In [1]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 SUPER


#installing dependencies

In [2]:
!pip install optuna==2.3.0
!pip install transformers==4.2.1
!pip install farasapy
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert

  Using cached transformers-4.2.1-py3-none-any.whl (1.8 MB)
  Using cached tokenizers-0.9.4-cp38-cp38-win_amd64.whl (1.9 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1


fatal: destination path 'arabert' already exists and is not an empty directory.


In [3]:
!pip install openpyxl
!pip install xlrd

#Creating training datasets

In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
all_datasets= []

In [3]:
class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

##HARD - Balanced

In [4]:
df_HARD = pd.read_excel('ML1.xlsx', engine='openpyxl', header=None)
df_HARD = df_HARD.rename(columns={0: 'tags', 1: 'post'})
print(df_HARD["tags"].value_counts())
# code rating as +ve if > 3, -ve if less, no 3s in dataset

train_HARD, test_HARD = train_test_split(df_HARD, test_size=0.2, random_state=42)
label_list_HARD = list(range(0, 5))

data_Hard = Dataset("ML1", train_HARD, test_HARD, label_list_HARD)
all_datasets.append(data_Hard)

4    730
3    369
2    325
1    303
0    252
Name: tags, dtype: int64


In [5]:
for x in all_datasets:
  print(x.name) 

ML1


#Trainer

In [27]:
!pip install transformers -U

In [6]:
from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
import optuna 

In [7]:
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [8]:
for x in all_datasets:
  print(x.name)

ML1


You can choose which model, and dataset from here along with the max sentence length

In [9]:
dataset_name = 'ML1'
model_name = 'aubmindlab/bert-base-arabertv2'
task_name = 'classification'
max_len = 256

In [10]:
for d in all_datasets:
  if d.name==dataset_name:
    selected_dataset = d
    print('Dataset found')
    break

Dataset found


In [11]:
arabert_prep = ArabertPreprocessor("bert-base-arabertv2")

selected_dataset.train['post'] = selected_dataset.train['post'].apply(lambda x: arabert_prep.preprocess(x))
selected_dataset.test['post'] = selected_dataset.test['post'].apply(lambda x: arabert_prep.preprocess(x))  

[2022-06-28 20:41:52,821 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
<ipython-input-11-db95ccf0e1e4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_dataset.train['post'] = selected_dataset.train['post'].apply(lambda x: arabert_prep.preprocess(x))
<ipython-input-11-db95ccf0e1e4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_dataset.test['post'] = selected_dataset.test['post'].apply(la

In [13]:
class BERTDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

In [19]:
label_map = { v:index for index, v in enumerate(selected_dataset.label_list) }
print(label_map)
train_dataset = BERTDataset(selected_dataset.train['post'].to_list(),selected_dataset.train['tags'].to_list(),model_name,max_len,label_map)
test_dataset = BERTDataset(selected_dataset.test['post'].to_list(),selected_dataset.test['tags'].to_list(),model_name,max_len,label_map)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4}


In [20]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [15]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  #print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))

  macro_f1_per_label = f1_score(p.label_ids, preds,average='macro', labels=list(range(0, 5)))
  macro_f1 = f1_score(p.label_ids,preds, average='macro')
  macro_precision = precision_score(p.label_ids, preds,average='macro')
  macro_recall = recall_score(p.label_ids, preds, average='macro')
  acc = accuracy_score(p.label_ids, preds)
  return {
      'macro_f1' : macro_f1,
      'macro_f1_per_label' : macro_f1_per_label,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
  }

# Regular Training

This paert allows you to do a regular training with no hyper parameter optimization

In [21]:
training_args = TrainingArguments(
    output_dir= "./train",
    do_eval = True,
    do_train = True,
    evaluation_strategy = 'epoch',
    adam_epsilon = 1e-8,
    learning_rate = 5e-5,
    fp16 = False,
    per_device_train_batch_size =16,
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps = 2,
    num_train_epochs= 8,
    warmup_ratio =0,
    save_strategy = 'epoch',
    seed = 42,
    lr_scheduler_type = 'cosine'
)

In [22]:
training_args.n_gpu

2

In [23]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


Some weights of the model checkpoint at aubmindlab/bert-base-arabertv2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

In [24]:
trainer.train()

C:\Users\bwfch\.conda\envs\bert\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1583
  Num Epochs = 8
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 2
  Total optimization steps = 200
  0%|          | 0/200 [00:00<?, ?it/s]C:\Users\bwfch\.conda\envs\bert\lib\site-packages\torch\nn\parallel\_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\Users\bwfch\.conda\envs\bert\lib\site-packages\torch\cuda\nccl.py:16: UserWarning: PyTorch is not compile

{'eval_loss': 0.14836037158966064, 'eval_macro_f1': 0.9758879023774458, 'eval_macro_f1_per_label': 0.9758879023774458, 'eval_macro_precision': 0.9684938691980947, 'eval_macro_recall': 0.9849139188794863, 'eval_accuracy': 0.9772727272727273, 'eval_runtime': 3.0096, 'eval_samples_per_second': 131.579, 'eval_steps_per_second': 4.32, 'epoch': 1.0}


Model weights saved in ./train\checkpoint-25\pytorch_model.bin
C:\Users\bwfch\.conda\envs\bert\lib\site-packages\torch\nn\parallel\_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\Users\bwfch\.conda\envs\bert\lib\site-packages\torch\cuda\nccl.py:16: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
 25%|██▌       | 50/200 [01:06<02:46,  1.11s/it]***** Running Evaluation *****
  Num examples = 396
  Batch size = 32

 25%|██▌       | 50/200 [01:09<02:46,  1.11s/it]Saving model checkpoint to ./train\checkpoint-50
Configuration saved in ./train\checkpoint-50\config.json


{'eval_loss': 0.07757890969514847, 'eval_macro_f1': 0.9843262475675848, 'eval_macro_f1_per_label': 0.9843262475675848, 'eval_macro_precision': 0.980004910009981, 'eval_macro_recall': 0.9889407645170702, 'eval_accuracy': 0.9848484848484849, 'eval_runtime': 2.982, 'eval_samples_per_second': 132.797, 'eval_steps_per_second': 4.36, 'epoch': 2.0}


Model weights saved in ./train\checkpoint-50\pytorch_model.bin
C:\Users\bwfch\.conda\envs\bert\lib\site-packages\torch\nn\parallel\_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\Users\bwfch\.conda\envs\bert\lib\site-packages\torch\cuda\nccl.py:16: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
 38%|███▊      | 75/200 [01:42<02:17,  1.10s/it]***** Running Evaluation *****
  Num examples = 396
  Batch size = 32

 38%|███▊      | 75/200 [01:45<02:17,  1.10s/it]Saving model checkpoint to ./train\checkpoint-75
Configuration saved in ./train\checkpoint-75\config.json


...

Model weights saved in ./train\checkpoint-175\pytorch_model.bin
C:\Users\bwfch\.conda\envs\bert\lib\site-packages\torch\nn\parallel\_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\Users\bwfch\.conda\envs\bert\lib\site-packages\torch\cuda\nccl.py:16: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
100%|██████████| 200/200 [04:47<00:00,  1.11s/it]***** Running Evaluation *****
  Num examples = 396
  Batch size = 32

100%|██████████| 200/200 [04:50<00:00,  1.11s/it]Saving model checkpoint to ./train\checkpoint-200
Configuration saved in ./train\checkpoint-200\config.json


{'eval_loss': 0.07642978429794312, 'eval_macro_f1': 0.9812157028157028, 'eval_macro_f1_per_label': 0.9812157028157028, 'eval_macro_precision': 0.9809412774118658, 'eval_macro_recall': 0.9815873942223519, 'eval_accuracy': 0.9823232323232324, 'eval_runtime': 3.0149, 'eval_samples_per_second': 131.347, 'eval_steps_per_second': 4.312, 'epoch': 8.0}


Model weights saved in ./train\checkpoint-200\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 200/200 [04:55<00:00,  1.48s/it]

{'train_runtime': 295.3835, 'train_samples_per_second': 42.873, 'train_steps_per_second': 0.677, 'train_loss': 0.13046462059020997, 'epoch': 8.0}


TrainOutput(global_step=200, training_loss=0.13046462059020997, metrics={'train_runtime': 295.3835, 'train_samples_per_second': 42.873, 'train_steps_per_second': 0.677, 'train_loss': 0.13046462059020997, 'epoch': 8.0})

In [25]:
trainer.save_model("types")

Saving model checkpoint to types
Configuration saved in types\config.json
Model weights saved in types\pytorch_model.bin


In [27]:
train_dataset.tokenizer.save_pretrained("types")

tokenizer config file saved in types\tokenizer_config.json
Special tokens file saved in types\special_tokens_map.json


('types\\tokenizer_config.json',
 'types\\special_tokens_map.json',
 'types\\vocab.txt',
 'types\\added_tokens.json',
 'types\\tokenizer.json')